In [1]:
import math
import time
import random

## Helper Functions

In [2]:
def scalarVectorProduct(scalar, vector):
    """
    This function multiplies a scalar with a vector.

    Parameters
    ----------
    scalar : float
        Scalar.
    vector : list
        List of numbers.
    """

    output = []
    for i in range(len(vector)):
        output.append(scalar * vector[i])

    return output

In [3]:
def addTwoVectors(vector1, vector2):
    """
    This function adds two vectors.

    Parameters
    ----------
    vector1 : list
        List of numbers.
    vector2 : list
        List of numbers.
    """

    output = []
    for i in range(len(vector1)):
        output.append(vector1[i] + vector2[i])

    return output

In [4]:
def transpose(matrix):
    """
    Transpose a matrix

    Parameters
    ----------
    matrix : list
        A list of lists representing a matrix
    """

    transpose_matrix = []
    for i in range(len(matrix[0])):
        row = []
        for j in range(len(matrix)):
            row.append(matrix[j][i])
        transpose_matrix.append(row)
    
    return transpose_matrix

In [5]:
def matrixVectorProduct(matrix, vector):
    """
    This function multiplies a matrix and a vector.

    Parameters
    ----------
    matrix : list
        A list of lists representing a matrix.
    vector : list
        A list representing a vector.
    """

    if len(matrix[0]) != len(vector):
        raise Exception("Invalid matrix dimensions")
    
    output = []
    for i in range(len(matrix)):
        summation = 0
        for j in range(len(vector)):
            summation += matrix[i][j] * vector[j]
        output.append(summation)
    
    return output

In [6]:
def matrixMultiplication(matrix1, matrix2):
    """
    Multiply two matrices

    Parameters
    ----------
    matrix1 : list
        A list of lists representing a matrix
    matrix2 : list
        A list of lists representing a matrix
    """

    shape1 = (len(matrix1), len(matrix1[0])) # shape1 = K x M1
    shape2 = (len(matrix2), len(matrix2[0])) # shape2 = M2 x N

    if shape1[1] != shape2[0]:               # if M1 != M2
        raise Exception("Invalid matrix dimensions")
    
    result = []
    for i in range(shape1[0]):              # iterate through rows of matrix1
        row = []                         
        for j in range(shape2[1]):          # iterate through columns of matrix2
            summation = 0
            for k in range(shape2[0]):      # calculate the inner product between row of matrix1 and column of matrix2
                summation += matrix1[i][k] * matrix2[k][j]
            row.append(summation)
        result.append(row)
    
    return result

In [7]:
def generateRandomMatrix(size, density, elements_range=(-5, 5)):
    """
    Generates a random matrix given the density of sparseness.
    
    Parameters
    ----------
    size : tuple 
        The size of the matrix.
    elements_range : tuple
        The range of the elements in the matrix.
    density : float
        The density of sparseness of the matrix.
        
    Returns
    -------
    matrix : list of lists
        The random sparse matrix.
    """
    matrix = []
    for i in range(size[0]):
        row = []
        for j in range(size[1]):
            # random.random() generates a number between 0 and 1 uniformly
            # if random.random() < density, then the element is non-zero otherwise it is zero
            if random.random() < density: 
                row.append(random.randint(elements_range[0], elements_range[1]))
            else:
                row.append(0)
        matrix.append(row)
    return matrix

In [8]:
def generateRandomPSDMatrix(size):
    """
    Generates a random positive semi-definite.

    Parameters
    ----------
    size : tuple 
        The size of the matrix.

    Returns
    -------
    matrix : list of lists
        The random positive semi-definite matrix.
    """

    matrix = generateRandomMatrix(size, 0.5)
    matrix = matrixMultiplication(transpose(matrix), matrix)
    return matrix

In [9]:
def printMatrix(matrix):
    """
    Prints the solution of the problem.
    
    Parameters
    ----------
    matrix : list of lists
        The matrix to be printed.
    """
    for i in range(len(matrix)):
        formatted_list = [ round(x, 2)  for x in matrix[i]]
        print(formatted_list)

In [10]:
def initialiseZeroMatrix(size):
    """
    This function creates a Zero Matrix.

    Parameters
    ----------
    size : int
        Size of the identity matrix.
    """

    output = [[0 for i in range(size)] for j in range(size)]
    return output

In [11]:
def checkTwoMatricesSame(matrix1, matrix2):
    """
    Checks if two matrices are the same.

    Parameters
    ----------
    matrix1 : list of lists
        The first matrix.
    matrix2 : list of lists
        The second matrix.
    
    Returns
    -------
    bool
        True if the matrices are the same, False otherwise.
    """

    if len(matrix1) != len(matrix2):
        return False
    
    for i in range(len(matrix1)):
        if len(matrix1[i]) != len(matrix2[i]):
            return False
        for j in range(len(matrix1[i])):
            if abs(matrix1[i][j] - matrix2[i][j]) > 1e-4:  # Floating point error
                return False
    
    return True

In [12]:
def printSolution(A, L):
    """
    Prints the solution of the problem.

    Parameters
    ----------
    A : list of lists
        The positive semidefinite matrix A.
    L : list of lists
        The lower triangular matrix L.
    """

    print("A:")
    printMatrix(A)
    print("-"*30)
    print("L:")
    printMatrix(L)
    print("-"*30)
    print("L^T")
    printMatrix(transpose(L))
    print("-"*30)
    print("L*L^T")
    A_hat = matrixMultiplication(L, transpose(L))
    printMatrix(A_hat)
    print("-"*30)
    print("Is A == L*L^T ?", checkTwoMatricesSame(A, A_hat))

## Main Code

In [25]:
def choleskyDecomposition(psd_matrix):
    """
    Performs Cholesky decomposition on a positive semi-definite matrix.

    Parameters
    ----------
    psd_matrix : list of lists
        The positive semi-definite matrix.
    
    Returns
    -------
    lower_triangular_matrix : list of lists
        The lower triangular matrix such that LL^T = psd_matrix.
    """

    lower_triangular_matrix = initialiseZeroMatrix(len(psd_matrix)) # Initialising lower_triangular matrix to identity matrix
    
    for k in range(len(psd_matrix)):
        summation_Lkj_squared = 0.0 

        for j in range(k):
            summation_Lki_Lji = sum([lower_triangular_matrix[k][i] * lower_triangular_matrix[j][i] for i in range(j)]) # summation(Lki*Lji) over i from 0 to j-1

            # Added 1e-5 to avoid division by zero error
            lower_triangular_matrix[k][j] = psd_matrix[k][j] - summation_Lki_Lji
            lower_triangular_matrix[k][j] /= (lower_triangular_matrix[j][j] + 1e-5) # Lkj = 1/Ljj(Akj - summation(Lki*Lji) over i from 0 to j-1) 
        
            summation_Lkj_squared += lower_triangular_matrix[k][j]**2 # summation(Lkj^2) over j from 0 to k-1
            
        lower_triangular_matrix[k][k] = math.sqrt(psd_matrix[k][k] - summation_Lkj_squared) # Lkk = sqrt(Akk - summation(Lkj^2)) over j from 0 to k-1
    
    return lower_triangular_matrix

In [26]:
A = generateRandomPSDMatrix((3, 3))
L = choleskyDecomposition(A)
printSolution(A, L)

A:
[10, 0, 6]
[0, 25, -20]
[6, -20, 20]
------------------------------
L:
[3.16, 0, 0]
[0.0, 5.0, 0]
[1.9, -4.0, 0.63]
------------------------------
L^T
[3.16, 0.0, 1.9]
[0, 5.0, -4.0]
[0, 0, 0.63]
------------------------------
L*L^T
[10.0, 0.0, 6.0]
[0.0, 25.0, -20.0]
[6.0, -20.0, 20.0]
------------------------------
Is A == L*L^T ? True
